# Préparation des Données - Performance des Étudiants

## Objectifs

Après avoir complété ce notebook, vous serez capable de :

- Gérer les valeurs manquantes dans les ensembles de données
- Corriger les formats de données
- Standardiser et normaliser les données
- Créer des groupes à partir de variables continues
- Générer des variables indicatrices à partir de données catégorielles en utilisant l'encodage one-hot

## Table des Matières

1. [Importation des Données et Bibliothèques](#import-data)
2. [Exploration des Données](#data-exploration)
3. [Gestion des Valeurs Manquantes](#missing-values)
4. [Correction des Formats de Données](#data-formats)
5. [Standardisation des Données](#standardization)
6. [Normalisation des Données](#normalization)
7. [Discrétisation des Données](#binning)
8. [Encodage One-Hot (Variables Indicatrices/Variables Dummy)](#indicator-variables)
9. [Exportation des Données Nettoyées](#export-data)

<a id="import-data"></a>
## 1. Importation des Données et Bibliothèques

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration du style de visualisation
sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
# Lecture du jeu de données
file_path = "StudentsPerformance.csv"
df = pd.read_csv(file_path)

# Affichage des premières lignes du jeu de données
df.head()

<a id="data-exploration"></a>
## 2. Exploration des Données

Avant de commencer le nettoyage et la préparation de nos données, comprenons d'abord ce avec quoi nous travaillons.

In [ ]:
# Obtenir des informations de base sur le jeu de données
print("Dimensions du jeu de données:", df.shape)
print("\nInformations sur le jeu de données:")
df.info()

In [ ]:
# Obtenir un résumé statistique du jeu de données
df.describe()

In [ ]:
# Vérifier les doublons
print(f"Nombre de lignes dupliquées: {df.duplicated().sum()}")

Explorons les variables catégorielles dans notre jeu de données :

In [ ]:
# Vérifier les valeurs uniques dans chaque colonne catégorielle
categorical_columns = ['gender', 'race/ethnicity', 'parents_education', 'lunch', 'test_prep_course']

for column in categorical_columns:
    print(f"\nValeurs uniques dans {column}:")
    print(df[column].value_counts())

<a id="missing-values"></a>
## 3. Gestion des Valeurs Manquantes

Vérifions s'il y a des valeurs manquantes dans notre jeu de données et traitons-les de manière appropriée.

In [ ]:
# Vérifier les valeurs manquantes
missing_values = df.isnull().sum()
print("Valeurs manquantes dans chaque colonne:")
print(missing_values)

Si nous trouvons des valeurs manquantes, nous pouvons les traiter en utilisant différentes méthodes :

1. **Supprimer les lignes avec des valeurs manquantes**
2. **Remplacer les valeurs manquantes par la moyenne, la médiane ou le mode**
3. **Remplacer les valeurs manquantes selon une stratégie spécifique**

Mettons en œuvre ces stratégies selon les besoins :

In [ ]:
# Exemple: S'il y avait des valeurs manquantes dans les colonnes numériques, nous pourrions les remplacer par la moyenne
numerical_columns = ['math_score', 'reading_score', 'writing_score']

for column in numerical_columns:
    if df[column].isnull().sum() > 0:
        mean_value = df[column].mean()
        df[column].fillna(mean_value, inplace=True)
        print(f"Valeurs manquantes remplacées dans {column} par la moyenne: {mean_value:.2f}")

In [ ]:
# Exemple: S'il y avait des valeurs manquantes dans les colonnes catégorielles, nous pourrions les remplacer par le mode
for column in categorical_columns:
    if df[column].isnull().sum() > 0:
        mode_value = df[column].mode()[0]
        df[column].fillna(mode_value, inplace=True)
        print(f"Valeurs manquantes remplacées dans {column} par le mode: {mode_value}")

<a id="data-formats"></a>
## 4. Correction des Formats de Données

Vérifions les types de données de nos colonnes et assurons-nous qu'ils sont au format correct.

In [ ]:
# Vérifier les types de données
df.dtypes

In [ ]:
# Convertir les types de données si nécessaire
# Par exemple, s'assurer que les colonnes numériques sont du type correct
for column in numerical_columns:
    df[column] = df[column].astype('float')

# Vérifier à nouveau les types de données
df.dtypes

<a id="standardization"></a>
## 5. Standardisation des Données

La standardisation des données est le processus de transformation des données en un format commun qui permet une comparaison significative.

In [ ]:
# Créons une nouvelle fonctionnalité : score moyen
df['average_score'] = (df['math_score'] + df['reading_score'] + df['writing_score']) / 3
df['average_score'] = df['average_score'].round(2)

# Afficher les premières lignes avec la nouvelle colonne
df.head()

In [ ]:
# Créer une colonne réussite/échec basée sur le score moyen (exemple de standardisation)
# En supposant que 60 est le score de passage
df['pass_status'] = np.where(df['average_score'] >= 60, 'Réussite', 'Échec')

# Afficher les premières lignes avec la nouvelle colonne
df.head()

<a id="normalization"></a>
## 6. Normalisation des Données

La normalisation est le processus de transformation des valeurs de plusieurs variables dans une plage similaire. Les techniques courantes de normalisation comprennent :

1. **Mise à l'échelle Min-Max** : Met les valeurs à l'échelle dans une plage fixe de 0 à 1
2. **Normalisation Z-score** : Standardise les valeurs en fonction de la moyenne et de l'écart-type

Appliquons ces techniques à nos données numériques :

In [ ]:
# Mise à l'échelle Min-Max
for column in numerical_columns:
    column_min = df[column].min()
    column_max = df[column].max()
    df[f'{column}_normalized'] = (df[column] - column_min) / (column_max - column_min)

In [ ]:
# Normalisation Z-score
for column in numerical_columns:
    mean = df[column].mean()
    std = df[column].std()
    df[f'{column}_standardized'] = (df[column] - mean) / std

Visualisons la distribution des scores originaux et normalisés :

In [ ]:
# Visualisation des données originales vs normalisées pour le score en mathématiques
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df['math_score'], kde=True)
plt.title('Scores Mathématiques Originaux')

plt.subplot(1, 3, 2)
sns.histplot(df['math_score_normalized'], kde=True)
plt.title('Scores Mathématiques Normalisés Min-Max')

plt.subplot(1, 3, 3)
sns.histplot(df['math_score_standardized'], kde=True)
plt.title('Scores Mathématiques Standardisés Z-Score')

plt.tight_layout()
plt.show()

<a id="binning"></a>
## 7. Discrétisation

La discrétisation est un processus de transformation des variables numériques continues en 'bins' catégoriels discrets pour une analyse groupée.

In [ ]:
# Regroupement des scores mathématiques en catégories
bins = [0, 40, 60, 80, 100]
labels = ['Faible', 'Moyen', 'Bon', 'Excellent']
df['math_score_category'] = pd.cut(df['math_score'], bins=bins, labels=labels, include_lowest=True)

# Afficher les premières lignes avec la colonne regroupée
df.head()

In [ ]:
# Compter le nombre d'étudiants dans chaque groupe
math_category_counts = df['math_score_category'].value_counts().sort_index()
print(math_category_counts)

In [ ]:
# Visualiser les données regroupées
plt.figure(figsize=(10, 6))
sns.countplot(x='math_score_category', data=df, palette='viridis')
plt.title('Distribution des Catégories de Scores en Mathématiques')
plt.xlabel('Catégorie de Score en Mathématiques')
plt.ylabel('Effectif')
plt.show()

<a id="indicator-variables"></a>
## 8. Encodage One-Hot (Variables Indicatrices/Variables Dummy)

L'encodage one-hot crée des colonnes binaires (0 ou 1) pour chaque catégorie dans les variables catégorielles. Ces variables sont également connues sous le nom de variables indicatrices ou variables dummy. Cette technique est essentielle pour inclure des données catégorielles dans des modèles statistiques qui nécessitent des entrées numériques.

In [ ]:
# Créer des variables dummy pour le genre en utilisant l'encodage one-hot
gender_dummies = pd.get_dummies(df['gender'], prefix='gender')
gender_dummies.head()

In [ ]:
# Créer des variables dummy pour la race/ethnicité en utilisant l'encodage one-hot
race_dummies = pd.get_dummies(df['race/ethnicity'], prefix='race')
race_dummies.head()

In [ ]:
# Créer des variables dummy pour le cours de préparation aux tests en utilisant l'encodage one-hot
test_prep_dummies = pd.get_dummies(df['test_prep_course'], prefix='test_prep')
test_prep_dummies.head()

In [ ]:
# Fusionner les variables dummy avec le dataframe original
df_with_dummies = pd.concat([df, gender_dummies, race_dummies, test_prep_dummies], axis=1)

# Afficher les premières lignes du dataframe avec les variables dummy
df_with_dummies.head()

Nous pouvons supprimer les colonnes catégorielles d'origine si nous voulons ne conserver que les variables dummy :

In [ ]:
# Créer une copie du dataframe pour cet exemple
df_encoded = df_with_dummies.copy()

# Supprimer les colonnes d'origine (conserver une version pour référence)
columns_to_drop = ['gender', 'race/ethnicity', 'test_prep_course']
df_encoded = df_encoded.drop(columns=columns_to_drop)

# Afficher les premières lignes du dataframe encodé
df_encoded.head()

<a id="export-data"></a>
## 9. Exportation des Données Nettoyées

Maintenant que nous avons nettoyé et transformé nos données, nous pouvons les exporter pour une analyse ultérieure.

In [ ]:
# Exporter le dataframe original avec les fonctionnalités ajoutées
df.to_csv('cleaned_students_performance_fr.csv', index=False)

# Exporter le dataframe avec les variables dummy
df_with_dummies.to_csv('students_performance_with_dummies_fr.csv', index=False)

# Exporter le dataframe entièrement encodé
df_encoded.to_csv('students_performance_encoded_fr.csv', index=False)

print("Données exportées avec succès !")

## Résumé

Dans ce notebook, nous avons :

1. Importé et exploré le jeu de données de performance des étudiants
2. Vérifié et traité les valeurs manquantes
3. Corrigé les formats de données
4. Standardisé les données en créant de nouvelles fonctionnalités
5. Normalisé les données numériques en utilisant la mise à l'échelle min-max et la standardisation z-score
6. Créé des groupes pour les variables continues
7. Généré des variables indicatrices pour les données catégorielles en utilisant l'encodage one-hot
8. Exporté les données nettoyées et transformées pour une analyse ultérieure

Ces techniques de préparation de données sont essentielles pour préparer les données brutes à l'analyse et à la modélisation.